<strong>Algorithme du voyageur de commerce (TSP)</strong>

In [2]:
import json
from pprint import pprint
# !pip3 install geopy
from geopy.distance import distance
import numpy as np
# !pip3 install fiona
# !pip3 install gdal
# !pip3 install proj
import geopandas as gpd
import pandas as pd

In [3]:
### comptons le nombre de features de notre jeu de données
with open('../data/reanimation.geojson') as f:
    data = json.load(f)

num_features = len(data['features'])

print("Le fichier GeoJSON reanimation.geojson contient", num_features, "features.")


Le fichier GeoJSON reanimation.geojson contient 297 features.


Il va falloir créer une matrice des distances entre chaque points. on va faire cela avec un csv et avec la formule de calcul de distance avec excel

In [4]:

# Créer une liste pour stocker les noms
names = []

# Boucle à travers les "features" dans le fichier GeoJSON
for feature in data['features']:
    # Vérifier si le champ "name" est présent dans le "feature"
    if 'name' in feature['properties']:
        name = feature['properties']['name']
        # Vérifier si le nom est déjà présent dans la liste
        if name in names:
            print("Le nom", name, "est en double.")
        else:
            names.append(name)

# Afficher un message si tous les noms sont uniques
if len(names) == len(data['features']):
    print("Tous les noms sont uniques.")


Tous les noms sont uniques.


In [5]:


# Lire le fichier GeoJSON en utilisant geopandas
gdf = gpd.read_file('../data/reanimation.geojson')

# Convertir en DataFrame Pandas
# df = gdf.drop('geometry', axis=1)

# Afficher les 5 premières lignes du DataFrame
pd.options.display.max_columns = None
gdf

,id,@id,CLC:id,addr:city,addr:country,addr:full,addr:housenumber,addr:place,addr:postcode,addr:street,addr:streetnumber,alt_name,alt_name_1,amenity,architect,architect:wikipedia,barrier,building,building:levels,capacity,children,contact:city,contact:facebook,contact:fax,contact:housenumber,contact:phone,contact:postcode,contact:street,contact:twitter,contact:website,description,dispensing,electricity,email,emergency,emergency:description,facebook,fax,foot,health_amenity:type,health_facility:type,healthcare,healthcare:speciality,height,heritage,heritage:operator,internet_access,internet_access:fee,internet_access:ssid,landuse,layer,level,loc_name,mapillary,medical_system:western,mhs:inscription_date,military,name,name:br,name:co,name:de,name:en,name:fr,name:pl,note,old_name,old_name:wikipedia,opening_hours,operator,operator:type,operator:wikidata,phone,ref:FR:FANTOIR,ref:FR:FINESS,ref:FR:NAF,ref:FR:SIREN,ref:FR:SIRET,ref:mhs,roof:levels,short_name,shortest_name,source,source:FINESS,source:addr,source:addr:housenumber:ref,source:addr:version,source:heritage,source:name:br,source:ref:FR:FINESS,start_date,survey:date,toilets:wheelchair,twitter,type,type:FR:FINESS,updated,url,wall,website,wheelchair,wikidata,wikipedia,@geometry,geometry
0,relation/214949,relation/214949,NaN,Lille,NaN,NaN,44,NaN,59000,Avenue Marx Dormoy,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+33 3 20 22 72 82,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DAPG3rfu50_hGnKMwcDwWg,NaN,NaN,NaN,Hôpital privé Le Bois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Polyclinique du Bois,NaN,NaN,Ramsay Santé,private,Q3123085,+33 3 59 75 75 75,NaN,590780268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-13,NaN,NaN,multipolygon,365,NaN,NaN,NaN,https://hopital-prive-le-bois-lille.ramsaygds.fr/,NaN,NaN,NaN,center,POINT (3.03119 50.63449)
1,relation/1221207,relation/1221207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Hospital of Paris,NaN,NaN,NaN,NaN,Hôpital américain de Paris,NaN,NaN,NaN,NaN,NaN,NaN,private,NaN,NaN,NaN,920008539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,365,NaN,NaN,NaN,NaN,NaN,Q470085,NaN,center,POINT (2.27242 48.89254)
2,relation/1227710,relation/1227710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital Jeanne de Flandre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,public,NaN,NaN,NaN,590006607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,center,POINT (3.03290 50.60574)
3,relation/1250218,relation/1250218,NaN,Lille,FR,NaN,NaN,NaN,59037,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital Roger Salengro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,public,NaN,NaN,NaN,590796975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,center,POINT (3.03289 50.60751)
4,relation/1471734,relation/1471734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital de Valence,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,+33 4 75 75 72 40,NaN,NaN,NaN,NaN,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre hospitalier de Valence,

In [6]:
### Afficher les colonnes qui n'ont pas de Nan
no_nans = []
for col in gdf.columns:
    if not gdf[col].isna().any():
        no_nans.append(col)

# Afficher les colonnes qui ne contiennent pas de NaN
print("Colonnes sans NaN : ", no_nans,"\n")

###afficher si il y a des lignes qui ont le même name
print("value_counts : \n",gdf.name.value_counts())

Colonnes sans NaN :  ['id', '@id', 'amenity', 'healthcare:speciality', 'name', 'ref:FR:FINESS', 'geometry'] 

value_counts : 
 Hôpital privé Le Bois                    1
Centre Hospitalier Louis Pasteur         1
Centre hospitalier de Roubaix            1
Centre hospitalier de Tourcoing          1
Hôpital Saint-Joseph                     1
                                        ..
Hôpital privé de Parly 2 Le Chesnay      1
Centre hospitalier de Mâcon              1
Centre hospitalier de Lons-le-Saunier    1
Centre hospitalier de Troyes             1
Hôpital Gui de Chauliac                  1
Name: name, Length: 297, dtype: int64


<strong>Résolution du TSP</strong>

Dans la pertie clustering, Paul a fait des clusters et calculé les coordonnées des centroïdes.

Ce qu'on va faire maintenant c'est calculer les tournées.

On va dans un premier temps calculer les distances à vol d'oiseau entre les points.

On va ensuite créer les tournées pour chaque cluster.

**Création de la matrice avec les distances à vol d'oiseau**

On récupère le csv qui contient les leiux et les coordonnées, ainsi que le numéro du cluster correspondant.

In [30]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from geopy.distance import great_circle
df = pd.read_csv('../Clustering/points+centroides.csv')
# Sélection des colonnes à utiliser pour calculer les distances
cols = ['name', 'latitude', 'longitude', 'cluster']
df_sub = df[cols]

# Extraction des noms de points
points = df_sub['name']

# Calcul de la matrice de distances
coords = df_sub[['latitude', 'longitude']].values
dist_matrix = squareform(pdist(coords, (lambda u,v: great_circle(u,v).km)))

# Création d'un dataframe à partir de la matrice de distances
dist_df = pd.DataFrame(dist_matrix, index=points, columns=points)

# Ajout des colonnes 'latitude', 'longitude' et 'cluster' à la matrice de distances
dist_df = df_sub[['name', 'latitude', 'longitude', 'cluster']].merge(dist_df, left_on='name', right_index=True)

# Affichage et exportation en CSV de la matrice de distances
dist_df.set_index('name', inplace=True)
dist_df.to_csv('distanceMatrice.csv', index=True)
dist_df


,latitude,longitude,cluster,Hôpital privé Le Bois,American Hospital of Paris,Hôpital Jeanne de Flandre,Hôpital Roger Salengro,Centre hospitalier de Valence,Hôpital Henri Mondor,Institut Calot,Hôpital de la Mère et de l’Enfant,Hôpital européen,Centre Henri Becquerel,Institut Universitaire du Cancer de Toulouse - Oncopole,Centre hospitalier Annecy-Genevois,Centre Hospitalier Saint-Joseph Saint-Luc,Hôpital Louis Mourier,Hôpital Foch,CHU Nord de Grenoble,Hôpital de la Cavale Blanche,Institut Coeur Poumon,Centre hospitalier de Niort,Hôpital Privé d'Antony,Clinique Mathilde,Centre Hospitalier de Narbonne,Hôpital Tenon,GHU Paris Psychiatrie & neurosciences - site Sainte-Anne,Fondation ophtalmologique Adolphe de Rothschild,Hôpital Cochin,Institut Mutualiste Montsouris,Hôpital Européen Georges Pompidou,Hôpital Armand-Trousseau,Hôpital Lariboisière,Centre Hospitalier de Roanne,Hôpital du Scorff,Centre hospitalier Notre-Dame de la Miséricorde,Centre hospitalier intercommunal de Créteil,Hôpital Purpan,Groupe hospitalier Pellegrin,Hôpital d'Instruction des Armées Robert Picqué,Centre hospitalier de Fontainebleau,Clinique du Millénaire,Centre hospitalier de Cholet,Hôpital Saint-Louis,Centre hospitalier de Perpignan,Hôpital Haut-Lévêque,Centre Chirurgical Marie Lannelongue,Centre Hospitalier Intercommunal Lucie et Raymond Aubrac,Hôpital Gabriel Montpied,Centre hospitalier de Meulan-en-Yvelines,Hôpital Sud,Hôpital Bretonneau,Hôpital Paul Brousse,Hôpital Robert Debré,Hôpital Bicêtre,Hôpital d'instruction des armées Sainte-Anne,Hôpital Avicenne,Centre hospitalier intercommunal Fréjus Saint-Raphaël,Centre hospitalier de Dieppe,Clinique d'Occitanie,Centre hospitalier de Longjumeau,Clinique de l'Union,Hôpital Émile Muller,Centre hospitalier de Pau,Centre hospitalier d'Albi,Hôpital de la Conception,Hôpital Simone Veil,Centre hospitalier intercommunal André Grégoire,Hôpital Jean Verdier,Hopital Beaujon,Centre hospitalier Comminges Pyrénées,Hôpital d'Instruction des Armées Bégin,Centre hospitalier,Hôpital Bichat-Claude Bernard,Centre hospitalier de Béziers,Centre hospitalier intercommunal de Cornouaille,Centre hospitalier intercommunal de Poissy,Hôpital Delafontaine,Centre hospitalier Jacques Lacarin,Hôpital Civil,Hôpital privé les Franciscaines,Centre hospitalier de Sambre-Avesnois,Centre Hospitalier Nord Ouest Villefranche,Hôpital de Hautepierre,Centre Hospitalier Régional Universitaire de Lille,Centre hospitalier de Cambrai,Hôpital Trousseau,Hôpital Central,Hôpital Charles Nicolle,Centre hospitalier intercommunal Robert Ballanger,Centre hospitalier de Mont-de-Marsan - Site de Sainte-Anne,Hôpial privé du Vert-Galant,Centre Hospitalier Henri Mondor,Centre hospitalier François Quesnay,Hôpital Ambroise Paré,Centre hospitalier des Pays de Morlaix,Hôpital Pasteur,Centre hospitalier de Haguenau,Centre hospitalier de Mont-de-Marsan,Centre hospitalier de Troyes,Centre hospitalier de Lons-le-Saunier,Centre hospitalier de Mâcon,Hôpital privé de Parly 2 Le Chesnay,Centre hospitalier de Bigorre - La Gespe,Polyclinique Bordeaux Nord Aquitaine,Centre hospitalier Ardèche Nord,Centre hospitalier de la Côte Basque - Site Cam de Prats,Centre hospitalier d’Auxerre,Centre hospitalier du Pays d'Aix,Centre hospitalier de Cahors,Clinique Mutualiste Chirurgicale,Hôpital Privé de la Loire,Hôpital Lozère,Centre hospitalier de Guéret,Hôpital Jacques Monod,Centre hospitalier d'Aubagne,Centre hospitalier universitaire de Limoges,Hôpital privé Jacques Cartier,Centre Hospitalier Sainte-Catherine,Clinique des Ormeaux,Centre hospitalier de Périgueux,Centre hospitalier de Versailles,Centre hospitalier de Creil,Centre hospitalier des vallées de l’Ariège - Site de Foix,Centre hospitalier de Moulins-Yzeure,Centre hospitalier de Lens,Hôpital Croix Saint-Simon,Hôpital privé La Casamance,Centre hospitalier Geneviève de Gaulle Anthonioz,Grand Hôpital de l'Est Francilien - Site de Marne-la-Vallée,Centre hospitalier de Brive-la-Gaillarde,Hôpital Saint-Philibert,Centre hospitalier de Meaux,C

**On a une matrice des distances, on va pouvoir maintenant créer les tournées pour chaque cluster**

In [229]:
dist_df = dist_df

# on va créer les tournées pour chaque cluster
centroide = pd.read_csv("../Clustering/centroids.csv").name # on crée un dataframe qui ne contient que les  centroides car ce sont els points de départ de chaque tournée

clusters = dist_df.cluster.unique()
def dictPoints(clusters = clusters):
    """crée un dictionnaire. chaque entrée est le numéro de cluster. chque entrée contient une liste des points qui sont dans le cluster.

    Returns:
        dict: clef = id cluster. entrée : liste des points de ce cluster
    """
    dicTournee = {}
    # Création d'une liste vide pour chaque cluster
    for elt in clusters:
        dicTournee[elt] = []
    
    #on ajoute chaque point dans son entrée de dictionnaire
    for idx, row in df.iterrows():
        cluster = row['cluster']
        name = row['name']
        if cluster in dicTournee:
            if name[0:9] != 'Centroïde':
                dicTournee[cluster].append(name)
    return dicTournee
    

def distance(point1, point2,df=dist_df):
    """
    point1 et point2 sont des str.\n
    renvoie la distance dans la matrice de distance entre le point1 et le point2.
    """
    distance = df[point1][point2]
    distance = int(round(float(distance)))
    return distance





def plus_proche_voisin(point1, points_restants):
    """
    Calcule le point le plus proche de point1, en augmentant la valeur minimale de la distance 
    tant que le point le plus proche et point1 n'ont pas le même cluster

    Args:
        point1 (str): Nom du point pour lequel on cherche le plus proche voisin
        df (DataFrame): DataFrame des distances
        points_restants (list): Liste des points restants à parcourir

    Returns:
        str: Nom du point le plus proche appartenant au même cluster que point1
    """
    dist=9999
    for elt in points_restants:
        if dist > distance(elt,point1):
            dist = distance(elt,point1)
            res = elt
    return res



def IdToCentroide(id, df=dist_df):
    """fonction qui prends un entier  le numéro de cluster et renvoie le centroide en str qui correspond

    Args:
        id (int): id du cluster correspondant au centroide
    
    Return (str) : le centoride correspondant au cluster
    """
    for index in df.index:
        if index == "Centroïde_"+str(id): # cherche l'index correspondant à l'id demandé
            return index


def tournees(idCentroide, seuil = 200, df = dist_df):
    """calcule les tournées pour le cluster donnés et les points qu'il doit parcourir.


    Args:
        centroide (int): le centroide correspondant au cluster et aux points.
        seuil (int) : nombre de km max que la gars peut faire en une tournée. 

    Returns : res (lis) : une liste de liste qui contient les tournées pour ce cluster
    """
    res=[]
    nameCentroide = IdToCentroide(idCentroide)
    liste_points = dictPoints(clusters = clusters)[idCentroide] # on récupère les ppoints que doit parcourir depuis ce centroide.
    liste_points_restants = liste_points.copy()
    while len(liste_points_restants) > 0 :

        tournee = [nameCentroide] # on initialise la tournée avec comme point de départ le centroide.
        distance_tournee = 0

        #on va chercher le point le plus proche
        proche = plus_proche_voisin(tournee[-1], points_restants = liste_points_restants) #on cherche le point le plus proche du avant dernier point de la tournée
        
        tournee.append(proche)
        liste_points_restants.remove(proche)

        distance_tournee += distance(point1=tournee[0], point2=tournee[-1])
        # print("distance_tournee : ",distance_tournee )
        # on va rajouter le point plus proche seulement si on ne dépasse pas le seuil de km par jour en comptant le retour au point d'arrivée.
        # print(tournee[-1], nameCentroide)
        while True:
            proche = plus_proche_voisin(point1= tournee[-1],  points_restants = liste_points_restants) #on cherche le point le plus proche du avant dernier point de la tournée
            # print("le point le plus proche de : ", tournee[-1], " est : ", proche)
            if (distance_tournee + distance(tournee[-1], proche)+distance(proche, nameCentroide)) < seuil:
                #on va ajouter le proche en avant dernière position de la tournée.
                distance_tournee+= distance(point1 = tournee[-1], point2 = proche) # ajoute à la distance de la tournée le nombre de km supplémentaire
                tournee.append(proche) #ajouter le point
                liste_points_restants.remove(proche) #supprimer le point qu'on vient d'ajouter de la liste des villes restantes. 

            else :
                tournee.append(nameCentroide)
                res.append(tournee)
                liste_points_restants.remove(proche)
                distance_tournee += distance(point1 = tournee[-1], point2 = nameCentroide)
                # print(res)

                break
            print("\n : len(liste_points_restants)", len(liste_points_restants))
    return res

def algoTSP():
    """calculer les tournées cluster par cluster
    """
    dict = {}
    for cluster in clusters:
        dict[cluster] = tournees(cluster, seuil = 600, df = dist_df)
    return dict

print(algoTSP())



 : len(liste_points_restants) 128

 : len(liste_points_restants) 127

 : len(liste_points_restants) 126

 : len(liste_points_restants) 125

 : len(liste_points_restants) 124

 : len(liste_points_restants) 123

 : len(liste_points_restants) 122

 : len(liste_points_restants) 121

 : len(liste_points_restants) 120

 : len(liste_points_restants) 119

 : len(liste_points_restants) 118

 : len(liste_points_restants) 117

 : len(liste_points_restants) 116

 : len(liste_points_restants) 115

 : len(liste_points_restants) 114

 : len(liste_points_restants) 113

 : len(liste_points_restants) 112

 : len(liste_points_restants) 111

 : len(liste_points_restants) 110

 : len(liste_points_restants) 109

 : len(liste_points_restants) 108

 : len(liste_points_restants) 107

 : len(liste_points_restants) 106

 : len(liste_points_restants) 105

 : len(liste_points_restants) 104

 : len(liste_points_restants) 103

 : len(liste_points_restants) 102

 : len(liste_points_restants) 101

 : len(liste_points

UnboundLocalError: local variable 'res' referenced before assignment

In [9]:
# !pip3 install _overlapped
# from asyncio.windows_events import NULL
from turtle import width


def donnees(nom_fichier):
    fichier = open(nom_fichier, "r")
    donnees = fichier.readlines()[1:]  # Ignorer la première ligne
    donnees = [t.strip().replace(",", ";") for t in donnees]
    donnees = [t.split(";") for t in donnees]
    fichier.close()
    print(type(donnees))
    print(donnees[2])
    return donnees

# print(temps_trajet)




def transcription_ville_indice(donnees): #affiche les villes une par une avec leur indice respectif (0,0 et 1, 0 et 1 et 2, etc....)
    ville_indice=dict()
    print(type(donnees))
    for i in range(len(donnees)):
        ville_indice[donnees[i][0]]=i
    return ville_indice

def transcription_indice_ville(donnees): #affiche l'ensemble des villes une fois avec leur indice
    indice_ville={}
    for i in range(len(donnees)):
        indice_ville[i]=donnees[i][0]
    return indice_ville

def distance(ville1,ville2): #retourne la distance en km entre deux villes
    i1=ville_indice[ville1]
    i2=ville_indice[ville2]
    imin=min(i1,i2)
    imax=max(i1,i2)
    return float(data[imax][imin+4])


def meilleur_tournee(liste_villes,nom_centroide): #retourne la liste des villes qu'il reste à parcourir avec un trajet optimal
    villes_restantes=liste_villes.copy()
    distance_totale = 0
    meilleur_tournee=[]
    ville_depart = nom_centroide
    meilleur_tournee.append(ville_depart)
    while len(villes_restantes)>0: #boucle qui s'arrête quand toutes les villes ont été parcouru
        ville_plus_proche=villes_restantes[0] #on débute en se disant que la ville la plus proche est la ville de dépar( logique !)
        ville_depart=meilleur_tournee[-1]
        distance_min=distance(ville_depart,ville_plus_proche) #calcule la distance entre la ville de départ et la ville la plus proche
        for ville in villes_restantes: #boucle qui parcourt les villes de villes_restantes
            if distance(ville_depart,ville)<distance_min and ville[0:10] != "Centroide_": #si la distance entre la ville de départ et la ville de l'itération est plus courte, alors la ville la plus proche devient la ville de l'itération
                ville_plus_proche=ville
                distance_min=distance(ville_depart,ville) #et la distance minimum devient la distance entre la ville de départ et la ville de l'itération
        if (distance_totale + distance_min + distance(ville_depart,meilleur_tournee[0]))*2 >=480:
            break
        distance_totale += distance_min
        villes_restantes.remove(ville_plus_proche) #à la fin de la boucle, il reste donc la ville la plus proche et on l'enlève donc de la liste villes_restantes
        meilleur_tournee.append(ville_plus_proche) #on l'ajoute ensuite à la liste meilleur trajet
    distance_totale += distance(ville_depart,meilleur_tournee[0])
    meilleur_tournee.append(meilleur_tournee[0]) #Puis on ajoute la ville de départ pour revenir au point de départ et faire une boucle
    villes_restantes = list(villes_restantes)
    print("\nLa distance de la tournée est ",distance_totale,"km !")
    print("len de meilelrue tour",len(meilleur_tournee))
    return [meilleur_tournee,villes_restantes,distance_totale]


def liste_ville(donnee): #retourne la liste de toute les villes du csv
    list = []
    list2 = []
    for i in range (0,len(donnee)-1,1):
        list.append(donnee[i])
        list2.append(list[i][0])
    return list2

def nom_ville(lat,long): #retourne la ville par rapport à des coordonnées (latitude et longitude)
    # donnee=donnees()
    ville_list=liste_ville(donnees)
    trajet=""
    for i in range (0,len(ville_list),1):
        if (float(donnees[i][1]) == lat and float(donnees[i][2]) == long): #float (ou autre) modifie l'élément en float
            ville = donnees[i][0]
            trajet = ville
    return trajet


temps_trajet=donnees("tempsTrajet.csv")
donnees_homebox = donnees("Homebox.csv")
data=donnees("distanceMatrice.csv")
tournee = 1 #numméro des tournées (augmente dans affichage_trajet)
somme_totale = 0 #la somme totale (augmente dans affichage_trajet)
ville_indice=transcription_ville_indice(data)
indice_ville=transcription_indice_ville(data)
print(ville_indice)
print(indice_ville)

list_magasin= liste_ville(data)
list_homebox = liste_ville(donnees_homebox)


def trajet_opti(num_tournee,somme_totale,list_magasin):
    print(len(list_magasin))
    trajet = []
    taille = 0
    # affichage_villes(list_magasin,True)
    # for i in range(0,len(list_magasin),1):
    while(len(list_magasin)>0):
        # if list_magasin[i-taille][0:11] == "Centroïde_0":
        print("caca")
        y = meilleur_tournee(list_magasin,"Centroide_0")
        taille += len(y[0])-2
        # affichage_trajet(y[0],somme_totale,num_tournee,True) #affiche les trajets
        trajet.append(y[0]) #ajoute dans trajet la première tournée
        print("La tournée est celle-ci:",y[0])
        list_magasin = y[1] #dit que le reste est toute les villes qui restent à parcourir
        num_tournee+=1 #ajoute +1 à la tournée car on vient de faire la première tournee
        somme_totale += y[2] #la somme totale est modifiée à chaque fin de tournée
    print("______list_magasin-----",len(list_magasin))
    print("\n La somme totale de la distance parcourue est de",somme_totale,"km !")
    print("Le trajet est celui-ci:",trajet,len(trajet))

    return [somme_totale,trajet]
trajet_opti(tournee,somme_totale,list_magasin)




<class 'list'>
['Hôpital Jeanne de Flandre', '50.6057446', '3.0329015', '0', '3.198970868935732', '198.17814293931139', '0.0', '0.19630512048314336', '648.1608400179656', '205.51370748851346', '105.97936942256928', '549.1239597330352', '830.3296061458773', '189.08138002173996', '793.1523688904294', '567.474485379671', '556.2291850043559', '195.4529406488748', '201.51372807694312', '633.9697535619106', '598.098434469002', '0.31688096980886155', '541.8666626203582', '212.1799667210341', '190.4431123016143', '825.5968240028825', '198.65190241508427', '203.8182319249791', '197.773074881242', '202.88228883388678', '204.55716093939714', '203.91032647197068', '201.22783361461464', '197.6948419217587', '513.2380912715188', '562.137954140015', '1059.4744406900409', '205.24604349899425', '787.6026534999492', '697.6385512118659', '699.6256593872534', '245.2304698708212', '781.6265971477779', '489.51248964962997', '198.423611939963', '876.3078665159013', '703.4392698832294', '209.7576026283598', '

KeyError: 'Centroide_0'